In [ ]:
'''
This code is for testing the dataframe decoding.
First we connect to the PMU, then ask for the cfg2 frame needed to decode the dataframe.
Read n measurements to test the transmission of the dataframes.
And last, we decode the dataframe
'''

In [1]:
import socket
import time
import math
import crcmod
from get_crc16 import get_crc16 #own function
from request_cfg2 import request_cfg2 #own function
from decode_cfg2 import decode_cfg2 #own function
from send_command import send_command #own function
#from decode_dataframe import decode_dataframe #own function


# Ask for dataframe

In [2]:
#PMU idcode
idcode=1
tcpip= "10.10.200.22"
tcpport = 4712
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.settimeout(5)
s.connect((tcpip, tcpport))


In [3]:
#ask for cfg2 to get the PMU parameters to use with the received dataframe
cfg2 = request_cfg2(s,idcode)


In [4]:
#send command to turn ON data transmission
cmd = 0b0010
send_command(s,idcode,cmd)

In [5]:
#read  measurements 
#this code is not optimal, n frames per second are expected
#is only for testing purposes, data could be lost or superimposed
n= 0
dataframenumber = 0 #counter for dataframes
#ask for 5 dataframes
while n  < 5:
    #can only recieve 4096 bytes
    #maybe i should use a function that can get me 
    #the numbers of bytes before using the recv function
    data = s.recv(4096)
    #check if first byte is AA (begining of the dataframe)
    if data[:1].hex() =='aa':
        #get the framesize
        framesizebytes  = data[2:4]
        framesize = int.from_bytes(framesizebytes,'big')
        #the dataframes could be superimposed, so i get the framesize and
        #divide the data into n framesize strings
        #split data in chunks of framsize
        if len(data) > framesize:
            datachunks = [data[i:i+n] for i in range(0, len(data), framesize)]
            for d in datachunks:
                print('Framesize: {}, length of dataframe: {}'.format(framesize,len(d)))
                print(d)
                #:-2 porque los ultimos bytes son el numero crc
                [_,_,crccalc] =get_crc16(d[:-2])
                crcsrc=int.from_bytes(d[-2:],'big')
                if crccalc==crcsrc:
                    print('CRC Ok!')
                else:
                    print('Error CRC')
                dataframenumber +=1
                
        else: #no superimposed dataframes
            print('Framesize: {}, length of dataframe: {}'.format(framesize,len(data)))
            print(data)
            dataframenumber +=1
            #:-2 porque los ultimos bytes son el numero crc
            [_,_,crccalc] =get_crc16(data[:-2])
            crcsrc=int.from_bytes(data[-2:],'big')
            if crccalc==crcsrc:
                print('CRC Ok!')
            else:
                print('Error CRC')
        print('Dataframe number: {}'.format(dataframenumber))
        print('------------------------------------------------------------')

    
#     print(len(data))
#     print('-----------------------------------------------------------------')
#     print(data.hex().upper())
    
    
    n=n+1
    
    


Framesize: 124, length of dataframe: 0
b''
Error CRC
Framesize: 124, length of dataframe: 0
b''
Error CRC
Framesize: 124, length of dataframe: 0
b''
Error CRC
Framesize: 124, length of dataframe: 0
b''
Error CRC
Framesize: 124, length of dataframe: 0
b''
Error CRC
Dataframe number: 5
------------------------------------------------------------
Framesize: 124, length of dataframe: 124
b'\xaa\x01\x00|\x00\x01\x00\x00\x0b\xf8\x0fLK@!p>\x93\x81\x1e@\x0c\xfc\xd1?\\\xa4l@\r\x14\x02;\xbc\x88\x01\xbe\xa8\x12q;.p\x00\xbe\xf1\x1eZ;\xd8\x18\x00\xbe\x9d\x94\xb99\x9a \x00\xbf\x85\xc0\xf4;fX\x00\xbe\xb6\xc2%:\x8f\x90\x00\xc0\x0c\xba\xc6BrF\x80B\x9b,\x95\xb4_;"5\'\xb5\x1f5\x1e&N\xbe\xaa`\x9c\xbb\x84\xaf\xfa;\xa1\xf0\x86;9\xab\xdf\xbfQ\xc1\xd4\x00\xff\xe0{'
CRC Ok!
Dataframe number: 6
------------------------------------------------------------
Framesize: 124, length of dataframe: 124
b'\xaa\x01\x00|\x00\x01\x00\x00\x0b\xf8\x0f[\x8d\x80!p>\x93\xa1\x10@\x0c\xedD?\\\xd4\xa8@\r\x04h;\xbc\x80\x01\xbe\xa8\

In [6]:
#send command to turn off data transmission
cmd = 0b0001
send_command(s,idcode,cmd)

In [7]:
s.close()

In [8]:
#need the configuration parameters from the cfg2 frame to decode dataframe
diccfg2 = decode_cfg2(cfg2,verbose=False)
time_base = diccfg2['TIME_BASE']
phasorNotation= diccfg2['PHASOR_notation']# polar o rectangular
phasorFormat = diccfg2['FORMAT_phasor']
phasorsnumber = diccfg2['PHNMR']#number of phasors
convFactor = diccfg2['PHUNIT_factor']
freqFormat = diccfg2['FORMAT_freq_dfreq']#float or int for freq/dfreq
analogFormat = diccfg2['FORMAT_analog']#float or int for analog
analogsNumber = diccfg2['ANNMR']# number of analog values
analogsNames = diccfg2['ANALOG_names']
phasorNames = diccfg2['PHASOR_names']
phasorUnit = diccfg2['PHUNIT_str']

# Decode dataframe

In [14]:
import struct
import math
from datetime import datetime

#####################################################
# 1 field SYNC
field1_1 = data[0] #it has to be 0xAA
if field1_1 != 0xAA:
    print('Transmission error: SYNC byte is {}'.format(field1_1.hex()))
field1_2 = data[1] #version number
ACKread = field1_1
print('ACK: {}'.format(hex(ACKread).upper()))
DATAFRAMETYPEread = field1_2 & 0b01110000 #bits 6-4
if DATAFRAMETYPEread == 0b000:
    strDATAFRAMETYPEread = 'Data Frame'

elif DATAFRAMETYPEread == 0b001:
    strDATAFRAMETYPEread = 'Header Frame'

elif DATAFRAMETYPEread == 0b010:
    strDATAFRAMETYPEread = 'Configuration Frame 1'

elif DATAFRAMETYPEread == 0b011:
    strDATAFRAMETYPEread = 'Configuration Frame 2'

elif DATAFRAMETYPEread == 0b100:
    strDATAFRAMETYPEread = 'Command Frame'
else:
    assert False, 'Decoding dataframe error: dataframe type {}'.format(DATAFRAMETYPEread)
print('Dataframe type: {}'.format(strDATAFRAMETYPEread))

VERread = field1_2 &0b00001111
print('Protocol version: {}'.format(VERread))
#######################################################
#2 FRAMESIZE
#it's always 18 because im not using the EXTFRAME data (table 12-command frame configuration)
[field2_1, field2_2] =data[2:4]
FRAMESIZEread = int.from_bytes([field2_1,field2_2],'big')
print('Framesize: {}'.format(FRAMESIZEread))
#######################################################
# 3 field IDCODE
#dividir en dos bytes y hacer un bitshift 8 lugares a la derecha
[field3_1, field3_2] = data[4:6]
IDCODEread = int.from_bytes([field3_1,field3_2],'big')
print('IDCODE: {}'.format(IDCODEread))
#######################################################
#4 SOC
[field4_1, field4_2, field4_3, field4_4] = data[6:10]
SOCread = int.from_bytes([field4_1,field4_2,field4_3,field4_4],'big')
print('SOC: {}'.format(SOCread))
#######################################################
#5 FRACSEC
[field5_1, field5_2, field5_3, field5_4] = data[10:14]
FRACSECread = int.from_bytes([field5_2,field5_3,field5_4],'big')
print('Fraction of second (raw): {}'.format(FRACSECread))

TIME_QUALITY = field5_1
TQbit7 = TIME_QUALITY & 128 #reserved
TQbit6 = TIME_QUALITY & 64 #leap second direction 0 for add, 1 for delete
#Leap second ocurred. Set in the first second after the leap
#second occurs and remains set for 24h.
TQbit5 = TIME_QUALITY & 32
#Leap second pending. Set before a leap second occurs and cleared
#in the second anfter the leap second occurs.
TQbit4 = TIME_QUALITY & 16
TQbits3_0 = TIME_QUALITY & 15
# decode time quality bits
if TQbit6 ==0:
        print('Leap second direction: False')
elif TQbit6 ==1:
        print('Leap second direction: True')
#probar que TQbit6 sea 0 o 1, si no manda un error
assert TQbit6   <=1, 'TQbit6 debe ser 0 o 1,no {}'.format(TQbit6)
if TQbit5 ==0:
        print('Leap second ocurred: False')
elif TQbit5 ==1:
        print('Leap second ocurred: True')
#probar que TQbit5 sea 0 o 1, si no manda un error
assert TQbit5   <=1, 'TQbit5 debe ser 0 o 1,no {}'.format(TQbit5)
if TQbit4 ==0:
        print('Leap second pending: False')
elif TQbit4 ==1:
        print('Leap second pending: True')
#probar que TQbit4 sea 0 o 1, si no manda un error
assert TQbit4   <=1, 'TQbit4 debe ser 0 o 1,no {}'.format(TQbit4)

if TQbits3_0 == 15:
    print('Fault-Clock failure, time not reliable.')
if TQbits3_0 == 11:
    print('Clock unlocked, time within 10 s.')
if TQbits3_0 == 10:
    print('Clock unlocked, time within 1 s.')
if TQbits3_0 == 9:
    print('Clock unlocked, time within 10E-1 s.')
if TQbits3_0 == 8:
    print('Clock unlocked, time within 10E-2 s.')
if TQbits3_0 == 7:
    print('Clock unlocked, time within 10E-3 s.')
if TQbits3_0 == 6:
    print('Clock unlocked, time within 10E-4 s.')
if TQbits3_0 == 5:
    print('Clock unlocked, time within 10E-5 s.')
if TQbits3_0 == 4:
    print('Clock unlocked, time within 10E-6 s.')
if TQbits3_0 == 3:
    print('Clock unlocked, time within 10E-7 s.')
if TQbits3_0 == 2:
    print('Clock unlocked, time within 10E-8 s.')
if TQbits3_0 == 1:
    print('Clock unlocked, time within 10E-9 s.')
if TQbits3_0 == 0:
    print('Normal operation, clock locked')

TIME_POSIX = SOCread + (FRACSECread/time_base);
#from unix timestamp to human readable time
TIMEread = datetime.utcfromtimestamp(TIME_POSIX).strftime('%Y-%m-%d %H:%M:%S.%f')
print('SOC time stamp: {} UTC'.format(TIMEread))
########################################################
#6 STAT
# Bitmapped flags.
# Bit 15: Data valid, 0 when PMU data is valid, 1 when invalid or PMU is in
# test mode.
# Bit 14: PMU error including configuration error, 0 when no error.
# Bit 13: PMU sync, 0 when in sync.
# Bit 12: Data sorting, 0 by time stamp, 1 by arrival.
# Bit 11: PMU trigger detected, 0 when no trigger.
# Bit 10: Configuration changed, set to 1 for 1 min when configuration
# changed.
# Bits 09–06: Reserved for security, presently set to 0.
# Bits 05–04: Unlocked time: 00 = sync locked, best quality
# 01 = Unlocked for 10 s
# 10 = Unlocked for 100 s
# 11 = Unlocked over 1000 s
# Bits 03–00: Trigger reason:
# 1111–1000: Available for user definition
# 0111: Digital 0110: Reserved
# 0101: df/dt high 0100: Frequency high/low
# 0011: Phase-angle diff 0010: Magnitude high
# 0001: Magnitude low 0000: Manual
STATread = data[14:16]
STATbit15 = (STATread[0] & 0b10000000) >> 7 
STATbit14 = (STATread[0] & 0b01000000) >> 6
STATbit13 = (STATread[0] & 0b00100000) >> 5
STATbit12 = (STATread[0] & 0b00010000) >> 4
STATbit11 = (STATread[0] & 0b00001000) >> 3
STATbit10 = (STATread[0] & 0b00000100) >> 2
STATbit09 = (STATread[0] & 0b00000010) >> 1
STATbit08 = STATread[0] & 0b00000001 

STATbit05_04 = (STATread[1] & 0b00110000) >> 4
STATbit03_00 = STATread[1] & 0b00001111

if STATbit15 == 0:
    print('STATbit15: {}. Data is valid.'.format(STATbit15))
elif STATbit15 ==1:
    print('STATbit15: {}. Data is not valid or PMU is in test mode'.format(
        STATbit15))

if STATbit14 == 0:
    print('STATbit14: {}. PMU error: No error'.format(STATbit14))
elif STATbit14 == 1:
    print('STATbit14: {}. PMU error: Error'.format(STATbit14))

if STATbit13 == 0:
    print('STATbit13: {}. Time synchronized: Sync'.format(STATbit13))
elif STATbit13 == 1:
    print('STATbit13: {}. Time synchronized: Time synchronization lost'.format(STATbit13))

if STATbit12 == 0:
    print('STATbit12: {}. Data sorting: by timestamp'.format(STATbit12))
elif STATbit12 == 1:
    print('STATbit12: {}. Data sorting: by arrival'.format(STATbit12))
    
if STATbit11 == 0:
    print('STATbit11: {}. Trigger detected: No Trigger'.format(STATbit11))
elif STATbit11 == 1:
    print('STATbit11: {}. Trigger detected: Yes'.format(STATbit11))

if STATbit10 == 0:
    print('STATbit10: {}. Configuration changed: No'.format(STATbit10))
elif STATbit10 == 1:
    print('STATbit10: {}. Configuration changed: Yes'.format(STATbit10))
    
if STATbit05_04 == 0b00:
    print('STATbit05_04: {}. sync locked, best quality'.format(STATbit05_04))
elif STATbit05_04 == 0b01:
    print('STATbit05_04: {}. Unlocked for 10 s'.format(STATbit05_04))
elif STATbit05_04 == 0b10:
    print('STATbit05_04: {}. Unlocked for 100 s'.format(STATbit05_04))
elif STATbit05_04 == 0b11:
    print('STATbit05_04: {}. Unlocked for over 1000 s'.format(STATbit05_04))
    
if STATbit03_00 == 0b0111:
    print('STATbit03_00: {}. Trigger reason: Digital'.format(STATbit03_00))
elif STATbit03_00 == 0b0101:
    print('STATbit03_00: {}. Trigger reason: df/dt high'.format(STATbit03_00))
elif STATbit03_00 == 0b0011:
    print('STATbit03_00: {}. Trigger reason: Phase-angle diff'.format(STATbit03_00))
elif STATbit03_00 == 0b0001:
    print('STATbit03_00: {}. Trigger reason: Magnitude low'.format(STATbit03_00))
elif STATbit03_00 == 0b0110:
    print('STATbit03_00: {}. Trigger reason NA: reserved bits'.format(STATbit03_00))
elif STATbit03_00 == 0b100:
    print('STATbit03_00: {}. Trigger reason: Frequency high/low'.format(STATbit03_00))
elif STATbit03_00 == 0b010:
    print('STATbit03_00: {}. Trigger reason: Magnitude high'.format(STATbit03_00))
elif STATbit03_00 == 0b000:
    print('STATbit03_00: {}. Trigger reason: Manual'.format(STATbit03_00))
elif STATbit03_00  > 0b1000 and STATbit03_00  <0b1111:
    print('STATbit03_00: {}. Trigger reason: user definition'.format(STATbit03_00))

##############################################
#PHASORS
#float type: 8 bytes
if phasorFormat == 'float':
    nbytes= 8
    phEndbyte = 16 + (nbytes*phasorsnumber) 
    phasors = data[16:phEndbyte]
    phasorMagnitude = []
    phasorAngle =[]
    if phasorNotation == 'polar':
        #first 4 bytes are the magnitude, next four bytes are the angle (i+4:i+8)
        for i in range(0,(phasorsnumber*nbytes),nbytes):
            #to convert bytestring to 32 bit floating point number
            phasorMagnitude.append(struct.unpack('>f', phasors[i:i+4])[0])
            #conversion to degrees
            phasorAngle.append(math.degrees( struct.unpack('>f', phasors[i+4:i+8])[0]))
        
    elif phasorNotation == 'rectangular':
        print('Rectangular notation not implemented yet')       

elif phasorFormat == 'int':
    print('Integer phasor format not implemented yet')


for i in range(phasorsnumber):
    print('Phasor #{}: {}, {:0.2f}{} {:0.2f}°'.format(
    i+1,phasorNames[i],phasorMagnitude[i],phasorUnit[i][0],phasorAngle[i]))
    
######################################################
#FREQ
if freqFormat == 'float':
    nbytes= 4
    freqEndbyte = phEndbyte + nbytes
    #to convert bytestring to 32 bit floating point number
    FREQread = struct.unpack('>f', data[phEndbyte:freqEndbyte])[0]

elif freqFormat =='int':
    #not tested yet
    nbytes= 2
    freqEndbyte = phEndbyte + nbytes
    #to convert bytestring to 32 bit floating point number
    FREQread = int.from_bytes(data[phEndbyte:freqEndbyte],'big')
###############################################################
#DFREQ
if freqFormat == 'float':
    nbytes= 4
    dFreqEndbyte = freqEndbyte + nbytes
    #to convert bytestring to 32 bit floating point number
    DFREQread = struct.unpack('>f', data[freqEndbyte:dFreqEndbyte])[0]

elif freqFormat =='int':
    #not tested yet
    nbytes= 2
    dFreqEndbyte = freqEndbyte + nbytes
    #to convert bytestring to 16 bits integer number
    DFREQread = int.from_bytes(data[freqEndbyte:dFreqEndbyte],'big')

#################################################################
#ANALOG
if analogFormat == 'float':
    nbytes = 4
    analogEndbyte = dFreqEndbyte + (nbytes * analogsNumber)
    ANALOGSread = data[dFreqEndbyte:analogEndbyte]
    ANALOGvalues = []
    for i in range(0,(analogsNumber*nbytes),nbytes):
        ANALOGvalues.append(struct.unpack('>f', ANALOGSread[i:i+nbytes])[0])
elif analogFormat == 'int':
    #not tested yet
    nbytes= 2
    analogEndbyte = dFreqEndbyte + (nbytes * analogsNumber)
    ANALOGSread = data[dFreqEndbyte:analogEndbyte]
    #to convert bytestring to 16 bits integer number
    ANALOGvalues = int.from_bytes(ANALOGSread[i:i+nbytes],'big')
for i,n in enumerate(analogsNames):
    print('Analog value #{}: {}, {:0.3f}'.format(
        i+1,n,round(ANALOGvalues[i],3)))

################################################################
#DIGITAL
nbytes = 2
digitalEndbyte = analogEndbyte + nbytes
DIGITALread = data[analogEndbyte:digitalEndbyte]

ACK: 0XAA
Dataframe type: Data Frame
Protocol version: 1
Framesize: 124
IDCODE: 1
SOC: 3064
Fraction of second (raw): 8000000
Leap second direction: False
Leap second ocurred: False
Leap second pending: False
Fault-Clock failure, time not reliable.
SOC time stamp: 1970-01-01 00:51:04.800000 UTC
STATbit15: 0. Data is valid.
STATbit14: 0. PMU error: No error
STATbit13: 1. Time synchronized: Time synchronization lost
STATbit12: 0. Data sorting: by timestamp
STATbit11: 0. Trigger detected: No Trigger
STATbit10: 0. Configuration changed: No
STATbit05_04: 3. Unlocked for over 1000 s
STATbit03_00: 0. Trigger reason: Manual
Phasor #1: b'V2+             ', 0.29V 127.43°
Phasor #2: b'VA2             ', 0.86V 127.52°
Phasor #3: b'IA2             ', 0.01A -17.59°
Phasor #4: b'VB2             ', 0.00V -25.75°
Phasor #5: b'IB2             ', 0.01A -16.39°
Phasor #6: b'VC2             ', 0.00V -58.68°
Phasor #7: b'IC2             ', 0.00A -19.27°
Phasor #8: b'IN2             ', 0.00A -124.48°
Analog 

In [13]:
datetime.utcfromtimestamp(TIME_POSIX).strftime('%Y-%m-%d %H:%M:%S.%f')


'1970-01-01 00:51:04.800000'

# test decode_dataframe function

In [2]:
import struct
import math
from datetime import datetime
def decode_dataframe(data,cfg2dict):
    #####################################################
    # 1 field SYNC
    field1_1 = data[0] #it has to be 0xAA
    if field1_1 != 0xAA:
        print('Transmission error: SYNC byte is {}'.format(field1_1.hex()))
    field1_2 = data[1] #version number
    ACKread = field1_1
    print('ACK: {}'.format(hex(ACKread).upper()))
    DATAFRAMETYPEread = field1_2 & 0b01110000 #bits 6-4
    if DATAFRAMETYPEread == 0b000:
        strDATAFRAMETYPEread = 'Data Frame'

    elif DATAFRAMETYPEread == 0b001:
        strDATAFRAMETYPEread = 'Header Frame'

    elif DATAFRAMETYPEread == 0b010:
        strDATAFRAMETYPEread = 'Configuration Frame 1'

    elif DATAFRAMETYPEread == 0b011:
        strDATAFRAMETYPEread = 'Configuration Frame 2'

    elif DATAFRAMETYPEread == 0b100:
        strDATAFRAMETYPEread = 'Command Frame'
    else:
        assert False, 'Decoding dataframe error: dataframe type {}'.format(DATAFRAMETYPEread)
    print('Dataframe type: {}'.format(strDATAFRAMETYPEread))

    VERread = field1_2 &0b00001111
    print('Protocol version: {}'.format(VERread))
    #######################################################
    #2 FRAMESIZE
    #it's always 18 because im not using the EXTFRAME data (table 12-command frame configuration)
    [field2_1, field2_2] =data[2:4]
    FRAMESIZEread = int.from_bytes([field2_1,field2_2],'big')
    print('Framesize: {}'.format(FRAMESIZEread))
    #######################################################
    # 3 field IDCODE
    #dividir en dos bytes y hacer un bitshift 8 lugares a la derecha
    [field3_1, field3_2] = data[4:6]
    IDCODEread = int.from_bytes([field3_1,field3_2],'big')
    print('IDCODE: {}'.format(IDCODEread))
    #######################################################
    #4 SOC
    [field4_1, field4_2, field4_3, field4_4] = data[6:10]
    SOCread = int.from_bytes([field4_1,field4_2,field4_3,field4_4],'big')
    print('SOC: {}'.format(SOCread))
    #######################################################
    #5 FRACSEC
    [field5_1, field5_2, field5_3, field5_4] = data[10:14]
    FRACSECread = int.from_bytes([field5_2,field5_3,field5_4],'big')
    print('Fraction of second (raw): {}'.format(FRACSECread))

    TIME_QUALITY = field5_1
    TQbit7 = TIME_QUALITY & 128 #reserved
    TQbit6 = TIME_QUALITY & 64 #leap second direction 0 for add, 1 for delete
    #Leap second ocurred. Set in the first second after the leap
    #second occurs and remains set for 24h.
    TQbit5 = TIME_QUALITY & 32
    #Leap second pending. Set before a leap second occurs and cleared
    #in the second anfter the leap second occurs.
    TQbit4 = TIME_QUALITY & 16
    TQbits3_0 = TIME_QUALITY & 15
    # decode time quality bits
    if TQbit6 ==0:
            print('Leap second direction: False')
    elif TQbit6 ==1:
            print('Leap second direction: True')
    #probar que TQbit6 sea 0 o 1, si no manda un error
    assert TQbit6   <=1, 'TQbit6 debe ser 0 o 1,no {}'.format(TQbit6)
    if TQbit5 ==0:
            print('Leap second ocurred: False')
    elif TQbit5 ==1:
            print('Leap second ocurred: True')
    #probar que TQbit5 sea 0 o 1, si no manda un error
    assert TQbit5   <=1, 'TQbit5 debe ser 0 o 1,no {}'.format(TQbit5)
    if TQbit4 ==0:
            print('Leap second pending: False')
    elif TQbit4 ==1:
            print('Leap second pending: True')
    #probar que TQbit4 sea 0 o 1, si no manda un error
    assert TQbit4   <=1, 'TQbit4 debe ser 0 o 1,no {}'.format(TQbit4)

    if TQbits3_0 == 15:
        print('Fault-Clock failure, time not reliable.')
    if TQbits3_0 == 11:
        print('Clock unlocked, time within 10 s.')
    if TQbits3_0 == 10:
        print('Clock unlocked, time within 1 s.')
    if TQbits3_0 == 9:
        print('Clock unlocked, time within 10E-1 s.')
    if TQbits3_0 == 8:
        print('Clock unlocked, time within 10E-2 s.')
    if TQbits3_0 == 7:
        print('Clock unlocked, time within 10E-3 s.')
    if TQbits3_0 == 6:
        print('Clock unlocked, time within 10E-4 s.')
    if TQbits3_0 == 5:
        print('Clock unlocked, time within 10E-5 s.')
    if TQbits3_0 == 4:
        print('Clock unlocked, time within 10E-6 s.')
    if TQbits3_0 == 3:
        print('Clock unlocked, time within 10E-7 s.')
    if TQbits3_0 == 2:
        print('Clock unlocked, time within 10E-8 s.')
    if TQbits3_0 == 1:
        print('Clock unlocked, time within 10E-9 s.')
    if TQbits3_0 == 0:
        print('Normal operation, clock locked')

    TIME_POSIX = SOCread + (FRACSECread/cfg2dict['TIME_BASE']);
    #from unix timestamp to human readable time
    TIMEread = datetime.utcfromtimestamp(TIME_POSIX).strftime('%Y-%m-%d %H:%M:%S')
    print('SOC time stamp: {} UTC'.format(TIMEread))
    ########################################################
    #6 STAT
    # Bitmapped flags.
    # Bit 15: Data valid, 0 when PMU data is valid, 1 when invalid or PMU is in
    # test mode.
    # Bit 14: PMU error including configuration error, 0 when no error.
    # Bit 13: PMU sync, 0 when in sync.
    # Bit 12: Data sorting, 0 by time stamp, 1 by arrival.
    # Bit 11: PMU trigger detected, 0 when no trigger.
    # Bit 10: Configuration changed, set to 1 for 1 min when configuration
    # changed.
    # Bits 09–06: Reserved for security, presently set to 0.
    # Bits 05–04: Unlocked time: 00 = sync locked, best quality
    # 01 = Unlocked for 10 s
    # 10 = Unlocked for 100 s
    # 11 = Unlocked over 1000 s
    # Bits 03–00: Trigger reason:
    # 1111–1000: Available for user definition
    # 0111: Digital 0110: Reserved
    # 0101: df/dt high 0100: Frequency high/low
    # 0011: Phase-angle diff 0010: Magnitude high
    # 0001: Magnitude low 0000: Manual
    STATread = data[14:16]
    STATbit15 = (STATread[0] & 0b10000000) >> 7
    STATbit14 = (STATread[0] & 0b01000000) >> 6
    STATbit13 = (STATread[0] & 0b00100000) >> 5
    STATbit12 = (STATread[0] & 0b00010000) >> 4
    STATbit11 = (STATread[0] & 0b00001000) >> 3
    STATbit10 = (STATread[0] & 0b00000100) >> 2
    STATbit09 = (STATread[0] & 0b00000010) >> 1
    STATbit08 = STATread[0] & 0b00000001

    STATbit05_04 = (STATread[1] & 0b00110000) >> 4
    STATbit03_00 = STATread[1] & 0b00001111

    if STATbit15 == 0:
        print('STATbit15: {}. Data is valid.'.format(STATbit15))
    elif STATbit15 ==1:
        print('STATbit15: {}. Data is not valid or PMU is in test mode'.format(
            STATbit15))

    if STATbit14 == 0:
        print('STATbit14: {}. PMU error: No error'.format(STATbit14))
    elif STATbit14 == 1:
        print('STATbit14: {}. PMU error: Error'.format(STATbit14))

    if STATbit13 == 0:
        print('STATbit13: {}. Time synchronized: Sync'.format(STATbit13))
    elif STATbit13 == 1:
        print('STATbit13: {}. Time synchronized: Time synchronization lost'.format(STATbit13))

    if STATbit12 == 0:
        print('STATbit12: {}. Data sorting: by timestamp'.format(STATbit12))
    elif STATbit12 == 1:
        print('STATbit12: {}. Data sorting: by arrival'.format(STATbit12))

    if STATbit11 == 0:
        print('STATbit11: {}. Trigger detected: No Trigger'.format(STATbit11))
    elif STATbit11 == 1:
        print('STATbit11: {}. Trigger detected: Yes'.format(STATbit11))

    if STATbit10 == 0:
        print('STATbit10: {}. Configuration changed: No'.format(STATbit10))
    elif STATbit10 == 1:
        print('STATbit10: {}. Configuration changed: Yes'.format(STATbit10))

    if STATbit05_04 == 0b00:
        print('STATbit05_04: {}. sync locked, best quality'.format(STATbit05_04))
    elif STATbit05_04 == 0b01:
        print('STATbit05_04: {}. Unlocked for 10 s'.format(STATbit05_04))
    elif STATbit05_04 == 0b10:
        print('STATbit05_04: {}. Unlocked for 100 s'.format(STATbit05_04))
    elif STATbit05_04 == 0b11:
        print('STATbit05_04: {}. Unlocked for over 1000 s'.format(STATbit05_04))

    if STATbit03_00 == 0b0111:
        print('STATbit03_00: {}. Trigger reason: Digital'.format(STATbit03_00))
    elif STATbit03_00 == 0b0101:
        print('STATbit03_00: {}. Trigger reason: df/dt high'.format(STATbit03_00))
    elif STATbit03_00 == 0b0011:
        print('STATbit03_00: {}. Trigger reason: Phase-angle diff'.format(STATbit03_00))
    elif STATbit03_00 == 0b0001:
        print('STATbit03_00: {}. Trigger reason: Magnitude low'.format(STATbit03_00))
    elif STATbit03_00 == 0b0110:
        print('STATbit03_00: {}. Trigger reason NA: reserved bits'.format(STATbit03_00))
    elif STATbit03_00 == 0b100:
        print('STATbit03_00: {}. Trigger reason: Frequency high/low'.format(STATbit03_00))
    elif STATbit03_00 == 0b010:
        print('STATbit03_00: {}. Trigger reason: Magnitude high'.format(STATbit03_00))
    elif STATbit03_00 == 0b000:
        print('STATbit03_00: {}. Trigger reason: Manual'.format(STATbit03_00))
    elif STATbit03_00  > 0b1000 and STATbit03_00  <0b1111:
        print('STATbit03_00: {}. Trigger reason: user definition'.format(STATbit03_00))

    ##############################################
    #PHASORS
    #float type: 8 bytes
    if cfg2dict['FORMAT_phasor'] == 'float':
        nbytes= 8
        phEndbyte = 16 + (nbytes*cfg2dict['PHNMR'])
        phasors = data[16:phEndbyte]
        phasorMagnitude = []
        phasorAngle =[]
        if cfg2dict['PHASOR_notation'] == 'polar':
            #first 4 bytes are the magnitude, next four bytes are the angle (i+4:i+8)
            for i in range(0,(cfg2dict['PHNMR']*nbytes),nbytes):
                #to convert bytestring to 32 bit floating point number
                phasorMagnitude.append(struct.unpack('>f', phasors[i:i+4])[0])
                #conversion to degrees
                phasorAngle.append(math.degrees( struct.unpack('>f', phasors[i+4:i+8])[0]))

        elif cfg2dict['PHASOR_notation'] == 'rectangular':
            print('Rectangular notation not implemented yet')

    elif cfg2dict['FORMAT_phasor'] == 'int':
        print('Integer phasor format not implemented yet')


    for i in range(cfg2dict['PHNMR']):
        print('Phasor #{}: {}, {:0.2f}{} {:0.2f}°'.format(
            i+1,cfg2dict['PHASOR_names'][i],phasorMagnitude[i],
                cfg2dict['PHUNIT_str'][i][0],phasorAngle[i]))
    ######################################################
    #FREQ
    if cfg2dict['FORMAT_freq_dfreq'] == 'float':
        nbytes= 4
        freqEndbyte = phEndbyte + nbytes
        #to convert bytestring to 32 bit floating point number
        FREQread = struct.unpack('>f', data[phEndbyte:freqEndbyte])[0]

    elif cfg2dict['FORMAT_freq_dfreq'] =='int':
        #not tested yet
        nbytes= 2
        freqEndbyte = phEndbyte + nbytes
        #to convert bytestring to 32 bit floating point number
        FREQread = int.from_bytes(data[phEndbyte:freqEndbyte],'big')
    ###############################################################
    #DFREQ
    if cfg2dict['FORMAT_freq_dfreq'] == 'float':
        nbytes= 4
        dFreqEndbyte = freqEndbyte + nbytes
        #to convert bytestring to 32 bit floating point number
        DFREQread = struct.unpack('>f', data[freqEndbyte:dFreqEndbyte])[0]

    elif cfg2dict['FORMAT_freq_dfreq'] == 'int':
        #not tested yet
        nbytes= 2
        dFreqEndbyte = freqEndbyte + nbytes
        #to convert bytestring to 16 bits integer number
        DFREQread = int.from_bytes(data[freqEndbyte:dFreqEndbyte],'big')

    #################################################################
    #ANALOG
    if cfg2dict['FORMAT_analog'] == 'float':
        nbytes = 4
        analogEndbyte = dFreqEndbyte + (nbytes * cfg2dict['ANNMR'])
        ANALOGSread = data[dFreqEndbyte:analogEndbyte]
        ANALOGvalues = []
        for i in range(0,(cfg2dict['ANNMR']*nbytes),nbytes):
            ANALOGvalues.append(struct.unpack('>f', ANALOGSread[i:i+nbytes])[0])
    elif cfg2dict['FORMAT_analog'] == 'int':
        #not tested yet
        nbytes= 2
        analogEndbyte = dFreqEndbyte + (nbytes * cfg2dict['ANNMR'])
        ANALOGSread = data[dFreqEndbyte:analogEndbyte]
        #to convert bytestring to 16 bits integer number
        ANALOGvalues = int.from_bytes(ANALOGSread[i:i+nbytes],'big')
    for i,n in enumerate(cfg2dict['ANALOG_names']):
        print('Analog value #{}: {}, {:0.3f}'.format(
            i+1,n,round(ANALOGvalues[i],3)))

    ################################################################
    #DIGITAL
    nbytes = 2
    digitalEndbyte = analogEndbyte + nbytes
    DIGITALread = data[analogEndbyte:digitalEndbyte]


In [2]:
#idcode de la PMU
idcode=1
tcpip= "10.10.200.22"
tcpport = 4712
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.settimeout(5)
s.connect((tcpip, tcpport))

#ask for cfg2 to get the PMU parameters to use with the received dataframe
cfg2 = request_cfg2(s,idcode)

#send command to turn ON data transmission
cmd = 0b0010
send_command(s,idcode,cmd)

#read  measurements 
#this code is not optimal, n frames per second are expected
#is only for testing purposes, data could be lost or superimposed
n= 0
dataframenumber = 0 #counter for dataframes
#ask for 5 dataframes
while n  < 5:
    #can only recieve 4096 bytes
    #maybe i should use a function that can get me 
    #the numbers of bytes before using the recv function
    data = s.recv(4096)
    #check if first byte is AA (begining of the dataframe)
    if data[:1].hex() =='aa':
        #get the framesize
        framesizebytes  = data[2:4]
        framesize = int.from_bytes(framesizebytes,'big')
        #the dataframes could be superimposed, so i get the framesize and
        #divide the data into n framesize strings
        #split data in chunks of framsize
        if len(data) > framesize:
            datachunks = [data[i:i+n] for i in range(0, len(data), framesize)]
            for d in datachunks:
                print('Framesize: {}, length of dataframe: {}'.format(framesize,len(d)))
                print(d)
                #:-2 porque los ultimos bytes son el numero crc
                [_,_,crccalc] =get_crc16(d[:-2])
                crcsrc=int.from_bytes(d[-2:],'big')
                if crccalc==crcsrc:
                    print('CRC Ok!')
                else:
                    print('Error CRC')
                dataframenumber +=1
                
        else: #no superimposed dataframes
            print('Framesize: {}, length of dataframe: {}'.format(framesize,len(data)))
            print(data)
            dataframenumber +=1
            #:-2 porque los ultimos bytes son el numero crc
            [_,_,crccalc] =get_crc16(data[:-2])
            crcsrc=int.from_bytes(data[-2:],'big')
            if crccalc==crcsrc:
                print('CRC Ok!')
            else:
                print('Error CRC')
        print('Dataframe number: {}'.format(dataframenumber))
        print('------------------------------------------------------------')

    
#     print(len(data))
#     print('-----------------------------------------------------------------')
#     print(data.hex().upper())
    
    
    n=n+1
    

#send command to turn off data transmission
cmd = 0b0001
send_command(s,idcode,cmd)
s.close()    


Framesize: 124, length of dataframe: 124
b'\xaa\x01\x00|\x00\x01\x00\x00\t,\x0f=\t\x00!p=Q\xdd\xe9\xc0\x17\x15\x96>\x1c\x93\xc0\xc0\x16m\xba;\x94\xe8\x00?L7n:\xe0 \x00?O\xb7e;\xaf\xe8\x00?M\xdfl9\x15\xc0\x00\xc0 \x12\xf0;7H\x00?Ogh:\xb2\x90\x00\xc0\x18\xdf\xafBlQ\x9e\xc2\x1b\xbd73\x03&\xfa3\x04\xe5N\xb1\xab\xa5\xe0?|\xa4:\xba3\xd8\x0b:3\xd9 6\x9d\xc8\xa4\xbf\x7f\xfev\x00\xffu\x0f'
CRC Ok!
Dataframe number: 1
------------------------------------------------------------
Framesize: 124, length of dataframe: 124
b'\xaa\x01\x00|\x00\x01\x00\x00\t,\x0fLK@!p=Q\xc3\xde\xc0\x1ag~>\x1c\x80\x10\xc0\x19\xc0A;\x94\xd8\x00?>\xdej:\xdf\x88\x00?B~c;\xaf\xc4\x00?@ff9\x12\x00\x00\xc0#+x;78\x00?A\xdec:\xb2\xc0\x00\xc0\x1c7\xeeBlM1\xc2\x1bQ\xc54\xf1f\xa34\xf1\xe7y2\xf9\x89H?\x7fw\xa8\xba3\xaf\x1f:3\xb0"6\x98\x86\xa4\xbf\x7f\xfe\x8f\x00\xff\xaa\r'
CRC Ok!
Dataframe number: 2
------------------------------------------------------------
Framesize: 124, length of dataframe: 124
b'\xaa\x01\x00|\x00\x01\x00\x00

In [4]:
diccfg2=decode_cfg2(cfg2)
decode_dataframe(data,diccfg2)

ACK: 0XAA
Dataframe type: Data Frame
Protocol version: 1
Framesize: 124
IDCODE: 1
SOC: 2348
Fraction of second (raw): 8000000
Leap second direction: False
Leap second ocurred: False
Leap second pending: False
Fault-Clock failure, time not reliable.
SOC time stamp: 1970-01-01 00:39:08 UTC
STATbit15: 0. Data is valid.
STATbit14: 0. PMU error: No error
STATbit13: 1. Time synchronized: Time synchronization lost
STATbit12: 0. Data sorting: by timestamp
STATbit11: 0. Trigger detected: No Trigger
STATbit10: 0. Configuration changed: No
STATbit05_04: 3. Unlocked for over 1000 s
STATbit03_00: 0. Trigger reason: Manual
Phasor #1: b'V2+             ', 0.05V -136.63°
Phasor #2: b'VA2             ', 0.15V -136.04°
Phasor #3: b'IA2             ', 0.00A 44.30°
Phasor #4: b'VB2             ', 0.00V 45.08°
Phasor #5: b'IB2             ', 0.01A 44.65°
Phasor #6: b'VC2             ', 0.00V -144.94°
Phasor #7: b'IC2             ', 0.00A 45.10°
Phasor #8: b'IN2             ', 0.00A -138.16°
Analog value #1